## Example Notebook on training a InceptionV3 Encoder and LSTM Decoder Image Captioning Model with Attention
What to expect:
1. Application of custom data loaders
2. Application of image captioning model with adaptive attention (Finetune and Pertrained InceptionV3 and Resnet50 models)
3. Application of BLEU and ROGUE
4. Application of model training, validation and test
5. Application of model inference

- Note: Due to the limitation of relative imports in notebooks, the full codes from src/helper functions have been copy pasted here to prevent the need for running imports
- Proper python functions in src/main.py will be rely on imports

Image Size for InceptionV3 must be larger than (299,299)

In [ ]:
#Imports
from PIL import Image, ImageOps
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

### 1. Application of custom data loaders

#### Imports

In [ ]:
#Tokenizer to split sentences into a list of words
word_tokenizer = spacy.load("en_core_web_sm")


class Vocabulary():
  """
  Class to convert the captions to index sequential tensors

  Args:
    freq_threshold (int, optional): How many times a word has to appear in dataset before it can be added to the vocabulary. Defaults to 2
  """

  def __init__(self, freq_threshold:int=2):
    self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"} #index to sentence
    self.stoi = {"<PAD>": 0, "<SOS>":1, "<EOS>": 2, "<UNK>":3} #sentence to index
    self.freq_threshold = freq_threshold #threshold for adding a word to the vocab

  def __len__(self):
    return len(self.itos)

  @staticmethod
  def tokenizer_eng(text):
    #convert sentence to list of words
    return [tok.text.lower() for tok in word_tokenizer.tokenizer(text)] #convert sentence to words


  def build_vocabulary(self, sentence_list):
    frequencies = {}
    idx = 4 #0-3 are for special tokens

    for sentence in sentence_list:
      for word in self.tokenizer_eng(sentence): #convert sentence to words
        if word not in frequencies:
          frequencies[word] = 1
        else:
          frequencies[word] += 1

        if frequencies[word] == self.freq_threshold: #once met freq_threshold, add to vocab list
          self.stoi[word] = idx
          self.itos[idx] = word
          idx += 1

  def numericalize(self, text):
    tokenized_text = self.tokenizer_eng(text) #convert annnotations to labels by converting each word to the index inside the vocab, else UNK tag
    return [
        self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
        for token in tokenized_text
    ]

In [ ]:
class CustomDataset(Dataset):

    """
    Class to setup the custom Dataset for pyTorch Dataloader

    Args:
        Note: the order of the csv_file and root_dir are directly related, csv_file[0] contains captions for images in root_dir[0]
        
        csv_file (list): Lists of path to CSV files with annotations.
        root_dir (list): List of directory containing images.
        img_size (tuple, optional): Image size in the format (width, height), defaults to (256,256)
        transform (callable, optional): Optional torchvision transform to be applied on a sample, defaults to None
        freq_threshold (int, optional): Freq threshold for Vocabulary Class, defaults to 2
        vocabulary (Vocabulary, optional): Determines to use an existing vocabulary or create own, defaults to None
    Returns:
        image: transformed image
        labels: tensor object of the labels
        all_image_captions: list containing all the captions of the image
    """

    def __init__(self, csv_file:list, root_dir:list, img_size:tuple=(256,256), transform=None, freq_threshold=2, vocabulary=None):
        
        #dataframe with col name ['image_filename', 'image_captions'] from csv file
        self.annotations = pd.DataFrame()
        #list containing the int boundary on which image path to look at
        #list will containing the num of images in directory which is the boundary
        self.root_dir_boundary = []
        
        for idx, label_files in enumerate(csv_file): 
            labels = pd.read_csv(label_files, index_col=0) #remove index col
            self.annotations = pd.concat([self.annotations, labels], ignore_index=True) #merging annotations into 1 dataset

            #getting the image boundary on which idx belongs to which image file path
            if idx == 0: 
                self.root_dir_boundary.append(len(labels))
            else:
                #get the number of images in root directory and add with the previous to get the range of index that are in this filepath
                self.root_dir_boundary.append(self.root_dir_boundary[idx-1] + len(labels))

        self.root_dir = root_dir
        self.transform = transform
        self.img_size = img_size

        #initialise vocabulary
        if vocabulary == None:
            self.vocab = Vocabulary(freq_threshold)
            self.vocab.build_vocabulary(self.annotations.iloc[:,1].to_list()) #build vocab with all captions
        else:
            self.vocab = vocabulary

    def __len__(self):
        return len(self.annotations)


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.annotations.iloc[idx, 0] #Image name as column 0
        #finding the correct root directory filepath in the list does the image belong to
        image_dir_idx = 0
        while idx >= self.root_dir_boundary[image_dir_idx] and image_dir_idx < len(self.root_dir_boundary):
            image_dir_idx += 1

        img_path = f"{self.root_dir[image_dir_idx]}/{img_name}"

        image = Image.open(img_path)
        # image = ImageOps.pad(image, self.img_size) #resize image
        annotation = self.annotations.iloc[idx, 1] #Annotation as column 1
        
        #converting caption to index tensor
        numercalized_annotations = [self.vocab.stoi["<SOS>"]]
        numercalized_annotations += self.vocab.numericalize(annotation)
        numercalized_annotations.append(self.vocab.stoi["<EOS>"])

        #create list of all captions associated with the image (for BLEU & ROUGE score)
        all_img_captions = self.annotations[self.annotations['image_filename'] == img_name]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(numercalized_annotations), all_img_captions.iloc[:,1].to_list()


In [ ]:
class padAnnotations():
  """
  Collate function to pad all caption to the same length as max(len(caption)) in a batch

  Args:
    pad_idx (int): Index Label for the <PAD> token
    batch_first (boolean, optional): Decide if the dataset labels should be batch first
                                     Either returns (batch size, seq length) or (seq length, batch size)
  
  Returns:
    img: batch image object
    labels: batch of tensors of the captions, converted to the same length by adding <PAD>
    all_labels: batch of lists of the captions of the images
  """
  def __init__(self, pad_idx, batch_first = False):
    self.batch_first = batch_first
    self.pad_idx = pad_idx

  def __call__(self, batch):
    imgs = [item[0].unsqueeze(0) for item in batch] 
    imgs = torch.cat(imgs, dim=0)
    labels = [item[1] for item in batch]
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=self.batch_first, padding_value=self.pad_idx)
    all_labels = [item[2] for item in batch]

    return imgs, labels, all_labels

In [ ]:
def visualise_img_caption_data(dataloader, batch_size: int, dataset: CustomDataset, num_batches: int = 1, num_samples: int=9):
    """
    Function to visualise the dataset

    Args:
        dataloader (dataloader object): Pytorch dataloader object to visualise
        batch_size (int): Batch Size of dataloader
        dataset (CustomDataset): dataset used to create dataloader (required to get the vocabulary)
        num_batches (int, optional): How many batches to visualise, defaults to 1
        num_samples (int, optional): How many images per batch to visualise, defaults to 9

    Returns:
        Plot of image with its caption and the list of the captions to the image
    """

    for i in range(num_batches):
        train_features, train_labels, train_all_captions = next(iter(dataloader))
        if i == 0:  # Print shape
            print(f"Feature batch shape: {train_features.size()}")
            print(f"Labels batch shape: {train_labels.size()}")
            print(f"All captions batch size: {len(train_all_captions)}")

        for j in range(num_samples):
            #Prepare image to print
            img = train_features[j].squeeze()
            img = np.transpose(img.numpy(), (1, 2, 0))  #Convert from (channel, height, width) to (height, width, channel) for matplotlib

            #Transpose train_labels and handle out of range indices
            label = train_labels[:, j] if train_labels.size(0) != batch_size else train_labels[j]
            string_label = [dataset.vocab.itos[idx] for idx in label.tolist()]
            actual_caption = " ".join([token for token in string_label if token not in ['<PAD>', '<SOS>', '<EOS>']])

            #Create a new plot for each image
            fig, ax = plt.subplots(figsize=(5, 5))
            ax.imshow(img)
            ax.axis('off')
            ax.set_title(f'Caption: {actual_caption}\n\nAll Possible Captions:\n' + "\n".join(train_all_captions[j]), loc='left')

            plt.tight_layout()
            plt.show()


#### Dataset Generation

In [ ]:
image_size = (299,299)
batch_size = 64

#Define transforms for image preprocessing
train_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.TrivialAugmentWide(num_magnitude_bins=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225]),
])

#No data augmentation
eval_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Actual dataset for this notebook

#Create train dataset
dataset = CustomDataset(csv_file=['../input/Landscape/Train/Labels/Blip_Label_Clean.csv', '../input/Landscape/Train/Labels/Kosmos_Label_Clean.csv', '../input/FilteredFlicker/Train/Labels/Label.csv'],
                        root_dir=['../input/Landscape/Train/Images', '../input/Landscape/Train/Images', '../input/FilteredFlicker/Train/Images'],
                        transform=train_transform,
                        img_size=image_size)

#Create val and test dataset, note that
#1. transform uses eval transform with no data augmentation
#2. vocab is the original train dataset vocab (to map the correct index)
val_dataset = CustomDataset(csv_file=['../input/Landscape/Validation/Labels/Blip_Label_Clean.csv', '../input/Landscape/Validation/Labels/Kosmos_Label_Clean.csv', '../input/FilteredFlicker/Validation/Labels/Label.csv'],
                        root_dir=['../input/Landscape/Validation/Images', '../input/Landscape/Validation/Images', '../input/FilteredFlicker/Validation/Images'],
                        transform=eval_transform,
                        img_size=image_size,
                        vocabulary=dataset.vocab)

#only focusing on landscape image, please add flicker30k if u would like
test_dataset = CustomDataset(csv_file=['../input/Landscape/Test/Labels/Blip_Label_Clean.csv', '../input/Landscape/Test/Labels/Kosmos_Label_Clean.csv'],
                        root_dir=['../input/Landscape/Test/Images', '../input/Landscape/Test/Images'],
                        transform=eval_transform,
                        img_size=image_size,
                        vocabulary=dataset.vocab)


#create dataloaders, just update the dataset it is from
train_dataloader = DataLoader(dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        collate_fn = padAnnotations(
                            pad_idx = dataset.vocab.stoi["<PAD>"],
                            batch_first=True
                        ))

val_dataloader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        collate_fn = padAnnotations(
                            pad_idx = dataset.vocab.stoi["<PAD>"],
                            batch_first=True
                        ))

test_dataloader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        collate_fn = padAnnotations(
                            pad_idx = dataset.vocab.stoi["<PAD>"],
                            batch_first=True
                        ))

In [ ]:
visualise_img_caption_data(train_dataloader, batch_size, dataset, num_samples=3)

### 2. Application of image captioning with attention model
- Implementing a InceptionV3 and a Resnet50 Encoder model that accepts fine-tuned model to a Attention and LSTM Decoder

In [ ]:
class InceptionV3EncoderCNN(torch.nn.Module):
    """
    InceptionV3 CNN Encoder Model, feature extraction layer (mixed_7c) is always trainable

    Args:
        finetuned_model: Finetuned inceptionV3 model, else None
        train_cnn (bool, optional): Determines if the entire CNN model will be unfreeze and trained during the training. Defaults to False.
    """
    def __init__(self, finetuned_model, train_cnn:bool=False):
        super(InceptionV3EncoderCNN, self).__init__()
        if finetuned_model != None:
            self.inception = list(finetuned_model.children())[0]
        
        else:
            self.inception = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
        self.inception.aux_logits = False
        
        #Remove last classification layer
        self.inception.fc = torch.nn.Identity()

        #Variable that will hold the features
        self.features = None
        
        #Register the hook to capture features at output of last CNN layer
        self.inception.Mixed_7c.register_forward_hook(self.capture_features_hook)

        #Train the feature map, the rest depends on train_CNN
        for name, param in self.inception.named_parameters():
            if 'Mixed_7c' in name:
                param.requires_grad_(True)
            else:
                param.requires_grad_(train_cnn)


    def capture_features_hook(self, module, input, output):
        self.features = output #update feature 


    def forward(self, images):
        """
        Take images and return feature maps of size (batch, height*width)
        """
        _ = self.inception(images)  #Pass through the inception network
        batch, feature_maps, size_1, size_2 = self.features.size()  #self.features contain the feature map of size (batch size, 2048, 8,8)
        features = self.features.permute(0, 2, 3, 1)
        features = features.view(batch, size_1*size_2, feature_maps) #resize to (batch size, h*w, feature_maps)

        return features
        

In [ ]:
class ResNetEncoderCNN(torch.nn.Module):
    """
    ResNet50 CNN Encoder Model, feature extraction layer (fc) is always trainable

    Args:
        finetuned_model: Finetuned inceptionV3 model, else None
        train_cnn (bool, optional): Determines if the entire CNN model will be unfreeze and trained during the training
    """
    def __init__(self, finetuned_model, train_cnn:bool=False):
        super(ResNetEncoderCNN, self).__init__()

        if finetuned_model != None:
            resnet = list(finetuned_model.children())[0]
        else:
            resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        #Train the feature map, the rest depends on train_CNN
        for name, param in resnet.named_parameters(): 
            if "layer4.2.conv3.weight" in name:
                param.requires_grad_(True)
            else:
                param.requires_grad_(train_cnn)
        
        modules = list(resnet.children())[:-2]
        self.resnet = torch.nn.Sequential(*modules)
        
    def forward(self, images):
        """
        Take images and return feature maps of size (batch, height*width)
        """
        features = self.resnet(images)
        # first, we need to resize the tensor to be 
        # (batch, h*w, feature_maps)
        batch, feature_maps, size_1, size_2 = features.size()       
        features = features.permute(0, 2, 3, 1)
        features = features.view(batch, size_1*size_2, feature_maps)
       
        return features

In [ ]:
class BahdanauAttention(torch.nn.Module):
    """
    Adaptive Attention Module

    Args:
        feature_dim (int): Dimension of feature maps (h*w)
        hidden_dim (int): Dimension of hidden states
        output_dim (int, optional): Dimension of output, default to 1
    """
    def __init__(self, feature_dim:int, hidden_dim:int, output_dim:int = 1):
        super(BahdanauAttention, self).__init__()
         # fully-connected layer to learn first weight matrix Wa
        self.W_a = torch.nn.Linear(feature_dim, hidden_dim)
        # fully-connected layer to learn the second weight matrix Ua
        self.U_a = torch.nn.Linear(hidden_dim, hidden_dim)
        # fully-connected layer to produce score (output), learning weight matrix va
        self.v_a = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, features, hidden_state):
        """
        Args:
            features: image features from Encoder
            hidden_state: hidden state output for Decoder

        Returns:
            context: context vector with size (1,2048)
            atten_weights: probabilities of feature relevance 
        """
        #add additional dimension to a hidden (required for summation) 
        hidden_state = hidden_state.unsqueeze(1) #(batch size, 1, seq length)

        atten_1 = self.W_a(features) #(batch size, h*w, hidden_dim)
        atten_2 = self.U_a(hidden_state) #(batch size, 1, hidden_dim)

        #apply tangent to combine result from 2 fc layers
        atten_tan = torch.tanh(atten_1+atten_2)
        atten_score = self.v_a(atten_tan) #(batch size, hidden_dim)
        atten_weight = torch.nn.functional.softmax(atten_score, dim = 1) #get softmax probablilities

        #multiply each vector with its softmax score and sum to get attention context vector
        context = torch.sum(atten_weight * features,  dim = 1) #size of context equals to a number of feature maps
        atten_weight = atten_weight.squeeze(dim=2)
        
        return context, atten_weight

In [ ]:
class DecoderRNN(torch.nn.Module):
     """
     LSTM decoder model

     Args:
          feature_dim (int): Feature Map dimension (h*w)
          embed_size (int): Embedding dimension to embed words
          hidden_size (int): Hidden state dimension for LSTM
          vocab_size (int): Total number of unique vocab
          drop_prob (float, optional): Dropout layer probability, deafults to 0.5
          sample_temp (float, optional): Scale outputs before softmax to allow the model to be more picky as the differences are exaggerated. Defaults to 0.5
     """
     def __init__(self, feature_dim:int, embedding_dim:int, hidden_dim:int, vocab_size:int, drop_prob:float=0.5, sample_temp:float=0.5):
          super(DecoderRNN, self).__init__()
          
          self.feature_dim = feature_dim
          self.embedding_dim = embedding_dim
          self.hidden_dim = hidden_dim
          self.vocab_size = vocab_size
          self.sample_temp = sample_temp #scale the outputs b4 softmax

          #layers

          #embedding layer that turns words into index 
          self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
          #lstm layer that takes in feature + embedding (image + caption) and output hidden_dim
          self.lstm = torch.nn.LSTMCell(embedding_dim + feature_dim, hidden_dim)
          #fc linear layer that predicts next word
          self.fc = torch.nn.Linear(hidden_dim, vocab_size)
          #attention layer
          self.attention = BahdanauAttention(feature_dim, hidden_dim)
          #dropout layer
          self.drop = torch.nn.Dropout(p=drop_prob)
          #initialisation of fully-connected layers
          self.init_h = torch.nn.Linear(feature_dim, hidden_dim) #initiialising hidden state and cell memory using avg of feature
          self.init_c = torch.nn.Linear(feature_dim, hidden_dim)

     def init_hidden(self, features):
          """
          Initializes hidden state and cell memory using average feature vector
          Args:
               features: feature map of the image
          Returns:
               h0: initial hidden state (short-term memory)
               c0: initial cell state (long-term memory)
          """
          mean_annotations = torch.mean(features, dim = 1) #getting average of the features
          h0 = self.init_h(mean_annotations)
          c0 = self.init_c(mean_annotations)
          return h0, c0

     def forward(self, features, captions, device:str, sample_prob:float=0.2):
          """
          Args:
               features: feature map of image
               captions: true caption of image
               device (str): cuda or cpu
               sample_prob (float, optional): Probability for auto-regressive RNN where they train on RNN output rather than true layer, defaults to 0.2

          """
          embed = self.embeddings(captions)
          h,c = self.init_hidden(features)
          batch_size = captions.size(0) #captions: (batch size, seq length)
          seq_len = captions.size(1) 
          feature_size = features.size(1) #features: (batch size, size, 2048)

          #storage of outputs and attention weights of lstm
          outputs = torch.zeros(batch_size, seq_len, self.vocab_size).to(device)
          atten_weights = torch.zeros(batch_size, seq_len, feature_size).to(device)

          #scheduled sampling for training, using the models output to train instead of using the true output
          #autoregressive RNN training, only when length of seq > 1 (cannot be first word)
          for t in range(seq_len):
               s_prob = 0.0 if t==0 else sample_prob
               use_sampling = np.random.random() < s_prob

               if not use_sampling: #no sampling
                    word_embeddings = embed[:, t, :] #embedding until word t
               
               context, atten_weight = self.attention(features,h)
               inputs = torch.cat([word_embeddings, context], 1) #embed captions and features for next lstm state
               h, c = self.lstm(inputs, (h,c)) #pass through lstm
               output = self.fc(self.drop(h))
               
               if use_sampling: #using predicted word instead of true output
                    scaled_output = output/self.sample_temp #using scaling temp to amplify the values
                    #this way softmax will have a larger difference in values
                    #makes the model more selective of whats its picking 
                    
                    scoring = torch.nn.functional.log_softmax(scaled_output, dim=1)
                    top_idx = scoring.topk(1)[1]
                    word_embeddings = self.embeddings(top_idx).squeeze(1) #update word embeddings with predicted instead of actual
               
               #update results
               outputs[:,t,:] = output
               atten_weights[:, t, :] = atten_weight

          return outputs, atten_weights




In [ ]:
class InceptV3EncoderAttentionDecoder(torch.nn.Module):
    """
    InceptionV3 Encoder with Attention and LSTM Decoder

    Args:
        finetuned_model: Finetuned InceptionV3 model, else None
        feature_dim (int): Feature Map dimension (h*w)
        embedding_dim (int): Embedding dimension to embed words
        hidden_dim (int): Hidden state dimension for LSTM
        vocab_size (int): Total number of unique vocab
        device (str): cuda or cpu
        train_cnn (boolean, optional): Determines if inceptionCNN model is unfreezed. Defaults to False.
        drop_prob (float, optional): Dropout layer probability. Defaults to 0.5.
        sample_temp (float, optional): Scale outputs before softmax to allow the model to be more picky as the differences are exaggerated. Defaults to 0.5
    """
    def __init__(self, finetuned_model, feature_dim:int, embedding_dim:int, hidden_dim:int, vocab_size:int, device:str, train_cnn:bool=False, drop_prob:float=0.5, sample_temp:float=0.5):
        super(InceptV3EncoderAttentionDecoder, self).__init__()
        self.encoder = InceptionV3EncoderCNN(finetuned_model, train_cnn)
        self.decoder= DecoderRNN(feature_dim, embedding_dim, hidden_dim, vocab_size, drop_prob, sample_temp)
        self.sample_temp = sample_temp
        self.device = device

    def forward(self, image, captions):
        features = self.encoder(image)
        outputs, atten_weights = self.decoder(features, captions, self.device, self.sample_temp)
        return outputs, atten_weights
    

    #for inference
    def caption_image(self, image, vocabulary:Vocabulary, device:str, max_length:int=50):
        """
        Generate caption using a greedy algorithm based on image input

        Args:
            image: image input
            vocabulary (Vocabulary): Vocabulary to decode predictions
            device (str): cuda or cpu
            max_length (int, optional): Max length of generated captions. Defaults to 50.

        Returns:
            captions: string caption in a list
            atten_weights: probabilities of feature relevance 
        """
        self.encoder.eval()

        result_caption = []
        result_weights = []

        with torch.no_grad(): #no training
            input_word = torch.tensor(1).unsqueeze(0).to(device)
            result_caption.append(1)
            features = self.encoder(image)
            h, c = self.decoder.init_hidden(features)

            for _ in range(max_length):
                embedded_word = self.decoder.embeddings(input_word)
                context, atten_weight = self.decoder.attention(features, h)
                # input_concat shape at time step t = (batch, embedding_dim + context size)
                input_concat = torch.cat([embedded_word, context],  dim = 1)
                h, c = self.decoder.lstm(input_concat, (h,c))
                h = self.decoder.drop(h)
                output = self.decoder.fc(h) 
                scoring = torch.nn.functional.log_softmax(output, dim=1)
                top_idx = scoring[0].topk(1)[1]
                result_caption.append(top_idx.item())
                result_weights.append(atten_weight)
                input_word = top_idx

                if (len(result_caption) >= max_length or vocabulary.itos[input_word.item()] == "<EOS>"):
                    break

            return [vocabulary.itos[idx] for idx in result_caption], result_weights


In [ ]:
class ResNetEncoderAttentionDecoder(torch.nn.Module):
    """
    Resnet50 Encoder with Attention and LSTM Decoder

    Args:
        finetuned_model: Finetuned InceptionV3 model, else None
        feature_dim (int): Feature Map dimension (h*w)
        embedding_dim (int): Embedding dimension to embed words
        hidden_dim (int): Hidden state dimension for LSTM
        vocab_size (int): Total number of unique vocab
        device (str): cuda or cpu
        train_cnn (boolean, optional): Determines if inceptionCNN model is unfreezed. Defaults to False.
        drop_prob (float, optional): Dropout layer probability. Defaults to 0.5.
        sample_temp (float, optional): Scale outputs before softmax to allow the model to be more picky as the differences are exaggerated. Defaults to 0.5
    """
    def __init__(self, finetuned_model, feature_dim:int, embedding_dim:int, hidden_dim:int, vocab_size:int, device:str, train_cnn:bool=False, drop_prob:float=0.5, sample_temp:float=0.5):
        super(ResNetEncoderAttentionDecoder, self).__init__()
        self.encoder = ResNetEncoderCNN(finetuned_model, train_cnn)
        self.decoder= DecoderRNN(feature_dim, embedding_dim, hidden_dim, vocab_size, drop_prob, sample_temp)
        self.sample_temp = sample_temp
        self.device = device

    def forward(self, image, captions):
        features = self.encoder(image)
        outputs, atten_weights = self.decoder(features, captions, self.device, self.sample_temp)
        return outputs, atten_weights
    

    #for inference
    def caption_image(self, image, vocabulary:Vocabulary, device:str, max_length:int=50):
        """
        Generate caption using a greedy algorithm based on image input

        Args:
            image: image input
            vocabulary (Vocabulary): Vocabulary to decode predictions
            device (str): cuda or cpu
            max_length (int, optional): Max length of generated captions. Defaults to 50.

        Returns:
            captions: string caption in a list
            atten_weights: probabilities of feature relevance 
        """
        self.encoder.eval()

        result_caption = []
        result_weights = []

        with torch.no_grad(): #no training
            input_word = torch.tensor(1).unsqueeze(0).to(device)
            result_caption.append(1)
            features = self.encoder(image)
            h, c = self.decoder.init_hidden(features)

            for _ in range(max_length):
                embedded_word = self.decoder.embeddings(input_word)
                context, atten_weight = self.decoder.attention(features, h)
                # input_concat shape at time step t = (batch, embedding_dim + context size)
                input_concat = torch.cat([embedded_word, context],  dim = 1)
                h, c = self.decoder.lstm(input_concat, (h,c))
                h = self.decoder.drop(h)
                output = self.decoder.fc(h) 
                scoring = torch.nn.functional.log_softmax(output, dim=1)
                top_idx = scoring[0].topk(1)[1]
                result_caption.append(top_idx.item())
                result_weights.append(atten_weight)
                input_word = top_idx

                if (len(result_caption) >= max_length or vocabulary.itos[input_word.item()] == "<EOS>"):
                    break

            return [vocabulary.itos[idx] for idx in result_caption], result_weights


### 3. Application of BLEU, ROGUE and METEOR

#### Imports

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def get_bleu_score(predicted:list, references:list):
    """
    Calculate the bleu 1-4 score of a corpus

    Args: 
        predicted (list): List of each individual prediction
            eg. [
                    "Transformers Transformers are fast plus efficient", 
                    "Good Morning", 
                    "I am waiting for new Transformers"
                ] 

        references (list): Nested list of references of each prediction

            eg. [
                    [
                        "HuggingFace Transformers are quick, efficient and awesome", 
                        "Transformers are awesome because they are fast to execute"
                    ], 
                    [
                        "Good Morning Transformers", 
                        "Morning Transformers"
                    ], 
                    [
                        "People are eagerly waiting for new Transformer models", 
                         "People are very excited about new Transformers"
                    ]
                ]

    Returns:
        results (dictionary): Dictionary in the format of {"BLEU1":, "BLEU2": ,"BLEU3": ,"BLEU4": }
    """
    #Tokenize the predictions and references
    predicted = [pred.split() for pred in predicted]
    references = [[ref.split() for ref in refs] for refs in references]


    BLEU1 = corpus_bleu(references, predicted, weights=(1, 0, 0, 0))
    BLEU2 = corpus_bleu(references, predicted, weights=(0.5, 0.5, 0, 0))
    BLEU3 = corpus_bleu(references, predicted, weights=(0.33, 0.33, 0.33, 0))
    BLEU4 = corpus_bleu(references, predicted, weights=(0.25, 0.25, 0.25, 0.25))

    return {"BLEU1": BLEU1, "BLEU2": BLEU2, "BLEU3": BLEU3, "BLEU4": BLEU4}

In [ ]:
import evaluate

def get_rouge_score(predicted:list, references:list):
    """
    Calculate the rogue1, rogue2, rogueL, rogueLSum scores
        
    Args: 
        predicted (list): List of each individual prediction
            eg. [
                    "Transformers Transformers are fast plus efficient", 
                    "Good Morning", 
                    "I am waiting for new Transformers"
                ] 

        references (list): Nested list of references of each prediction

            eg. [
                    [
                        "HuggingFace Transformers are quick, efficient and awesome", 
                        "Transformers are awesome because they are fast to execute"
                    ], 
                    [
                        "Good Morning Transformers", 
                        "Morning Transformers"
                    ], 
                    [
                        "People are eagerly waiting for new Transformer models", 
                         "People are very excited about new Transformers"
                    ]
                ]

    Returns:
        results (dictionary): Dictionary in the format of {'rouge1': , 'rouge2': , 'rougeL': , 'rougeLsum': }
    
    """
    rouge = evaluate.load('rouge')
    results = rouge.compute(predictions=predicted, references=references)
    return results

In [ ]:
import evaluate

def get_meteor_score(predicted:list, references:list):
    """
    Calculate the meteor scores
        
    Args: 
        predicted (list): List of each individual prediction
            eg. [
                    "Transformers Transformers are fast plus efficient", 
                    "Good Morning", 
                    "I am waiting for new Transformers"
                ] 

        references (list): Nested list of references of each prediction

            eg. [
                    [
                        "HuggingFace Transformers are quick, efficient and awesome", 
                        "Transformers are awesome because they are fast to execute"
                    ], 
                    [
                        "Good Morning Transformers", 
                        "Morning Transformers"
                    ], 
                    [
                        "People are eagerly waiting for new Transformer models", 
                         "People are very excited about new Transformers"
                    ]
                ]

    Returns:
        results (dictionary): Dictionary in the format of {'meteor':}
    
    """
    meteor = evaluate.load('meteor')
    results = meteor.compute(predictions=predicted, references=references)
    return results

### 4. Application of model training, validation and test

#### Imports

In [ ]:
def decode_predictions(outputs, batch_first:bool, vocabulary:Vocabulary):
    """
    Function to convert model tensor outputs to sentences

    Args:
        outputs (torch tensor object): Model's output to be decoded, either in size (seq len, batch, vocab_size) or (batch, seq len, vocab_size)
        batch_first (bool): Boolean of if dataloader was configured to batch_first
        vocabulary (Vocabulary): dataset Vocabulary Class for decoding

    Returns:
        list of predicted sentences each corresponding to 1 sample in the batch
            - will be of length (batch_size)
            eg. ['predicted sentence 1 for sample 1', ...'predicted sentence N for sample N']
    
    """

    all_prediction = []
    predicted_tokens = outputs.argmax(-1) #flatten vocab size dimensions
    if not batch_first:
        predicted_tokens = predicted_tokens.T 
    
    for sentence_tokens in predicted_tokens:
        sentence_tokens = sentence_tokens.tolist()

        try:
            #cropping predicted sentence to first EOS
            eos_index = sentence_tokens.index(vocabulary.stoi['<EOS>']) #get first instance of <EOS> to crop sentence accordingly
            predicted_sentence = sentence_tokens[:eos_index]
        except:
            predicted_sentence = sentence_tokens

        try:
            #getting predicted_sentence by remove <SOS>
            predicted_sentence.remove(vocabulary.stoi['<SOS>'])
        except:
            pass
    
        all_prediction.append(" ".join([vocabulary.itos[idx] for idx in predicted_sentence]))

    return all_prediction

In [ ]:
def eval(model, 
         criterion, 
         dataloader, 
         image_size:tuple,
         transformer:bool,
         batch_first:bool, 
         vocabulary:Vocabulary, 
         device:str):
    """
    Function to evaluate model performance

    Args:
        model: The model that is to be evaluated
        criterion: Loss criterion of the model
        dataloader: validation / test dataset
        image_size (tuple): image size of model
        transformer (bool): boolean if decoder is a transformer
        batch_first (bool): boolean if dataloader samples tensor are (batch, seq len) or (seq len, batch)
        vocabulary (Vocabulary): dataset vocabulary class
        device (str): cpu or cuda

    Returns:
        avg_val_loss: average validation loss
        Bleu_score: dictionary of BLEU 1-4 score
        Rouge_score: dictionary of Rouge  1,2,L,LSum score
        Meteor_score: dictionary of Meteor Score
    """
    
    model.eval()

    total_val_loss = 0

    #BLEU predictions container
    predictions = []
    references = []
    
    with torch.no_grad():
        for idx, (imgs, annotations, all_annotations) in enumerate(dataloader):
            #getting img and annotations
            imgs = torch.nn.functional.interpolate(imgs, size=image_size, mode='bilinear') #resize image for model, using same as transforms.resize()
            imgs = imgs.to(device)
            annotations = annotations.to(device)
            
            if transformer:
                #running model prediction
                outputs = model(imgs, annotations[:-1]) #training model to guess the last word
                targets = annotations[1:].reshape(-1)
                #updating model parameters
                loss = criterion(outputs.view(-1, len(vocabulary)), targets)
            
            else:
                if not batch_first:
                    #running model prediction
                    outputs = model(imgs, annotations[:-1]) #training model to guess the last word
                    
                    #updating model parameters
                    loss = criterion(outputs.reshape(-1, outputs.shape[2]), annotations.reshape(-1)) #reshape output (seq_len, N, vocabulary_size) to (N, vocabulary_size)
                
                if batch_first:
                    #running model prediction
                    outputs, atten_weights = model(imgs, annotations[:, :-1]) #training model to guess the last word
                    targets = annotations[:, 1:]
                    #updating model parameters
                    loss = criterion(outputs.view(-1, len(vocabulary)), targets.reshape(-1)) #reshape output (seq_len, N, vocabulary_size) to (N, vocabulary_size)
            
            total_val_loss += loss.item()

            #get model predictions and update
            predictions.extend(decode_predictions(outputs, batch_first, vocabulary))

            #update references
            references.extend(all_annotations)

        Bleu_score = get_bleu_score(predictions, references)
        Rouge_score = get_rouge_score(predictions, references)
        Meteor_score = get_meteor_score(predictions, references)

        return total_val_loss/(idx+1), Bleu_score, Rouge_score, Meteor_score


In [ ]:
def train(model, 
          criterion, 
          optimiser, 
          train_dataloader, 
          val_dataloader, 
          image_size: tuple,
          batch_first:bool, 
          transformer:bool,
          vocabulary:Vocabulary, 
          device:str, 
          num_epochs:int, 
          early_stopping_threshold: float=0.003,
          show_train_metrics:bool=None, 
          save_every:int=None,
          model_name:str=None, 
          overwrite:bool=False):
    
    """
    Function to train the model
    If change in val loss is less than 0.01 for 2 epochs in a row, stop training

    Args:
        model: The model that is to be evaluated
        criterion: Loss criterion of the model
        optimiser: Optimiser function of the model
        train_dataloader: Train dataset
        val_dataloader: Validation dataset, use None if no Validation dataset
        image_size (tuple): image size of model
        batch_first (bool): Boolean if dataloader samples tensor are (batch, seq len) or (seq len, batch)
        transformer (bool): Boolean if decoder is a transformer
        vocabulary (Vocabulary): Dataset vocabulary class
        device (str): cpu or cuda
        num_epochs (int): Number of epochs for training
        early_stopping_threshold (float, optional): Threshold for early stopping, defaults to 0.003
        show_train_metrics (bool, optional): Booleon on should calculate BLEU & Rouge score during training, defaults to False
        save_every (int, optional): Save model after every ___ epochs, defaults to None (no saving)
        model_name (str, optional): Model Name to be saved after, required if save_every != None, model will be saved as (model_name)_epoch or just model_name
        overwrite (bool, optional): Boolean on overwriting model saves or saving each specific epoch as a new model, defaults to False
    
    Returns
        train_loss: list of average training loss per epoch
        train_bleu: list of dictionary of training BLEU score per epoch, [] if show_train_metric = False
        train_rouge: list of dictionary of training Rouge score per epoch, [] if show_train_metric = False
        train_meteor: list of dictionary of training Meteor score per epoch, [] if show_train_metric = False
        val_loss: list of average validation loss per epoch, [] if val_dataloader = None
        val_bleu: list of dictionary of validation BLEU score per epoch, [] if val_dataloader = None
        val_rouge: list of dictionary of validation Rouge score per epoch, [] if val_dataloader = None
        val_meteor: list of dictionary of validation Meteor score per epoch, [] if val_dataloader = None

    
    """
    
    #initialise results container
    train_loss = []
    train_bleu = []
    train_rouge = []
    train_meteor = []

    val_loss = []
    val_bleu = []
    val_rouge = []
    val_meteor = []

    for epoch in range(num_epochs):

        total_train_loss = 0

        #BLEU predictions container
        predictions = []
        references = []

        #start model training
        model.train()
        for idx, (imgs, annotations, all_annotations) in enumerate(train_dataloader):
            
            #getting img and annotations
            imgs = torch.nn.functional.interpolate(imgs, size=image_size, mode='bilinear') #resize image for model, using same as transforms.resize()
            imgs = imgs.to(device)
            annotations = annotations.to(device)

            if transformer:
                #running model prediction
                outputs = model(imgs, annotations[:-1]) #training model to guess the last word
                targets = annotations[1:].reshape(-1)
                #updating model parameters
                loss = criterion(outputs.view(-1, len(vocabulary)), targets)
            
            else:
                if not batch_first:
                    #running model prediction
                    outputs = model(imgs, annotations[:-1]) #training model to guess the last word
                    #updating model parameters
                    loss = criterion(outputs.reshape(-1, outputs.shape[2]), annotations.reshape(-1)) #reshape output (seq_len, N, vocabulary_size) to (N, vocabulary_size)
                
                if batch_first:
                    #running model prediction
                    outputs, atten_weights = model(imgs, annotations[:, :-1]) #training model to guess the last word
                    targets = annotations[:, 1:]
                    #updating model parameters
                    loss = criterion(outputs.view(-1, len(vocabulary)), targets.reshape(-1)) #reshape output (seq_len, N, vocabulary_size) to (N, vocabulary_size)
                
            optimiser.zero_grad() #remove optimiser gradient
            loss.backward()
            optimiser.step()
            
            #calculate loss and update it for each batch
            total_train_loss += loss.item()

            if show_train_metrics:
                #get model predictions and update
                predictions.extend(decode_predictions(outputs, batch_first, vocabulary))

                #update references
                references.extend(all_annotations)

        if show_train_metrics:   
            #calculating bleu and rouge score
            Bleu_score = get_bleu_score(predictions, references)
            Rouge_score = get_rouge_score(predictions, references)
            Meteor_score = get_meteor_score(predictions, references)
            train_bleu.append(Bleu_score)
            train_rouge.append(Rouge_score)
            train_meteor.append(Meteor_score)

        #updating values
        train_loss.append(total_train_loss/(idx+1))

        if val_dataloader != None:
            #validation
            avg_val_loss, val_bleu_score, val_rouge_score, val_meteor_score = eval(
                                                                            model=model,
                                                                            criterion=criterion,
                                                                            dataloader=val_dataloader,
                                                                            image_size=image_size,
                                                                            transformer=transformer,
                                                                            batch_first=batch_first,
                                                                            vocabulary=vocabulary,
                                                                            device=device
                                                                        )
            
            val_loss.append(avg_val_loss)
            val_bleu.append(val_bleu_score)
            val_rouge.append(val_rouge_score)
            val_meteor.append(val_meteor_score)

        #printing progress
        if num_epochs <= 10 or (num_epochs >10 and (epoch+1)%5 == 0):
            print(f"Epoch {epoch+1} completed\navg training loss per batch: {total_train_loss/(idx+1)}")
            
            if show_train_metrics:
                print(f"train bleu score:{Bleu_score}\ntrain rouge score: {Rouge_score}\ntrain meteor score: {Meteor_score}\n")

            if val_dataloader != None:
                print(f"avg validation loss per batch: {avg_val_loss}\nval bleu score: {val_bleu_score}\nval rouge score: {val_rouge_score}\nval meteor score: {val_meteor_score}")

            print("------------------------------------------------------------------")
            
        #saving model every x
        if save_every != None and (epoch+1)%save_every == 0:
            try:
                if overwrite:
                    torch.save(model.state_dict(), f"../models/image_captioning/{model_name}.pt")
                else:
                    torch.save(model.state_dict(), f"../models/image_captioning/{model_name}_{epoch+1}.pt")
            except:
                print(f"Unable to save model at epoch {epoch+1}")

        
        #saving best model
        if (len(val_loss) > 1) and val_loss[-1] < min(val_loss[:-1]):
            try:
                torch.save(model.state_dict(), f"../models/image_captioning/{model_name}_best.pt")
            except:
                print(f"Unable to save best model")
        

        #early stopping
        if (len(val_loss) >= 3) and abs(val_loss[-2] - val_loss[-1]) < early_stopping_threshold and abs(val_loss[-3] - val_loss[-2]) < early_stopping_threshold:
            print(f"validation loss did not decrease, stopping training at epoch {epoch +1}")
            try:
                if overwrite:
                    torch.save(model.state_dict(), f"../models/image_captioning/{model_name}.pt")
                else:
                    torch.save(model.state_dict(), f"../models/image_captioning/{model_name}_{epoch+1}.pt")
            except:
                print(f"Unable to save model at epoch {epoch+1}")
            break


    return train_loss, train_bleu, train_rouge, train_meteor, val_loss, val_bleu, val_rouge, val_meteor


In [ ]:
def visualise_graph(training_data:list, validation_data:list, y_label:str, x_label:str, ylim:list=None):
    """
    Plot a line graph against training and validation data

    Args:
        training_data (list): Training data to be plotted
        validation_data (list): Validation data to be plotted
        y_label (str): Label of y axis
        x_label (str): Label of x axis
        ylim (list, optional): Range of y axis, defaults to None
    """
    #plotting line graph of training data and validation data
    plt.plot(training_data, label='Train')
    plt.plot(validation_data, label='Validation')

    #labels and title
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(f'{y_label} against {x_label}')
    
    if ylim != None:
        plt.ylim(ylim)
        
    #show legend
    plt.legend()
    plt.show()
    

#### Training

In [ ]:
# Clear GPU memory
import gc
torch.cuda.empty_cache()
gc.collect()

##### Pretrained Model Imports

In [ ]:
class PreTrainedCNNModels(torch.nn.Module):
    def __init__(self, model_type:str, num_unfreeze:int, num_class:int):
        super(PreTrainedCNNModels, self).__init__()
        """
        Class that contains InceptionV3, Resnet50, Resnet152, EfficientNet, DenseNet, VGG16, MaxVit fine tuned models

        Args:
            model_type (str): Determines which pre-trained models to use
                              Must be: InceptionV3, Resnet50, Resnet152, EfficientNet, DenseNet, VGG16, MaxVit
            num_unfreeze (int): Number of layers to unfreeze and finetune
            num_class (int): Number of output classes for the classification
        """
        #selecting model type
        if model_type == 'InceptionV3':
            self.model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
            self.model.aux_logits = False

        elif model_type == 'Resnet50':
            self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

        elif model_type == 'Resnet152':
            self.model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)

        elif model_type == 'EfficientNet':
            self.model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)

        elif model_type == 'DenseNet':
            self.model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        
        elif model_type == 'VGG16':
            self.model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)

        elif model_type == 'MaxVit':
            self.model = models.maxvit_t(weights=models.MaxVit_T_Weights.DEFAULT)
        
        else:
            raise Exception("Invalid model type chosen. Please select one of the following\n[InceptionV3, Resnet50, Resnet152, EfficientNet, DenseNet, VGG16, MaxVit]")

        
        #modifying final layer
        if model_type in ['InceptionV3', 'Resnet50', 'Resnet152']:
            self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_class)

        elif model_type == 'DenseNet':
            self.model.classifier = torch.nn.Linear(self.model.classifier.in_features, num_class)

        else:
            self.model.classifier[-1] = torch.nn.Linear(self.model.classifier[-1].in_features, num_class)


        model_paramteres = list(self.model.parameters())
        #unfreeze last num_unfreeze layers
        for param in model_paramteres[-num_unfreeze:]:
            param.requires_grad = True

        #freeze rest of the layers
        for param in model_paramteres[:-num_unfreeze]:
            param.requires_grad = False


    def forward(self, images):
        return self.model(images)

##### Finetuned InceptionV3

In [ ]:
finetuned_incept = torch.load('../models/cnn/incept_12_best.pt')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyper parameters
img_size = (299,299)
feature_size = 2048
embed_size = 256 
hidden_size = 512
vocab_size = len(dataset.vocab)
learning_rate = 3e-4
num_epochs = 100
save_every = 25
model_name = "AttnModel_Incept12FT"


print(device)

In [ ]:
model = InceptV3EncoderAttentionDecoder(finetuned_model=finetuned_incept,
                                      feature_dim=feature_size,
                                      embedding_dim=embed_size, 
                                      hidden_dim=hidden_size, 
                                      vocab_size=vocab_size, 
                                      device=device,
                                      train_cnn=False,
                                      drop_prob=0.3,
                                      sample_temp=0.2).to(device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loss, train_bleu, train_rouge, train_meteor, val_loss, val_bleu, val_rouge, val_meteor=train(
    model=model,
    criterion=criterion,
    optimiser=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    image_size=img_size,
    transformer=False,
    batch_first=True,
    vocabulary=dataset.vocab,
    device=device,
    num_epochs=num_epochs,
    early_stopping_threshold = 0.001,
    show_train_metrics=True,
    save_every=save_every,
    model_name=model_name,
    overwrite=False
)

In [ ]:
#loss
visualise_graph(training_data=train_loss,
                validation_data=val_loss,
                y_label='Loss',
                x_label='Epoch')

#rouge-1
rouge_train =  [rouge_dict['rouge1'] for rouge_dict in train_rouge]
rouge_val =  [rouge_dict['rouge1'] for rouge_dict in val_rouge]
visualise_graph(training_data=rouge_train,
                validation_data=rouge_val,
                y_label='Rouge-1 Score',
                x_label='Epoch')

#bleu4
bleu4_train =  [bleu_dict['BLEU4'] for bleu_dict in train_bleu]
bleu4_val =  [bleu_dict['BLEU4'] for bleu_dict in val_bleu]
visualise_graph(training_data=bleu4_train,
                validation_data=bleu4_val,
                y_label='BLEU-4 Score',
                x_label='Epoch')

#meteor
meteor_train =  [meteor_dict['meteor'] for meteor_dict in train_meteor]
meteor_val =  [meteor_dict['meteor'] for meteor_dict in val_meteor]
visualise_graph(training_data=meteor_train,
                validation_data=meteor_val,
                y_label='Meteor Score',
                x_label='Epoch')

In [ ]:
# Save the data into csv
data = zip(train_loss, bleu4_train, rouge_train, meteor_train, val_loss, bleu4_val, rouge_val, meteor_val)
dataframe = pd.DataFrame(data, columns=["Train Loss","Train BLEU", "Train ROUGE", "Train METEOR","Val Loss", "Val BLEU", "Val ROUGE", "Val METEOR"])
dataframe.to_csv(f"../resources/training_results/{model_name}.csv")

##### Pretrained InceptionV3

In [ ]:
# Clear GPU memory
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyper parameters
img_size = (299,299)
feature_size = 2048
embed_size = 256 
hidden_size = 512
vocab_size = len(dataset.vocab)
learning_rate = 3e-4
num_epochs = 75
save_every = 25
model_name = "AttnModel_InceptPT"


print(device)

In [ ]:
model = InceptV3EncoderAttentionDecoder(finetuned_model=None,
                                      feature_dim=feature_size,
                                      embedding_dim=embed_size, 
                                      hidden_dim=hidden_size, 
                                      vocab_size=vocab_size, 
                                      device=device,
                                      train_cnn=False,
                                      drop_prob=0.3,
                                      sample_temp=0.2).to(device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loss, train_bleu, train_rouge, train_meteor, val_loss, val_bleu, val_rouge, val_meteor=train(
    model=model,
    criterion=criterion,
    optimiser=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    image_size=img_size,
    transformer=False,
    batch_first=True,
    vocabulary=dataset.vocab,
    device=device,
    num_epochs=num_epochs,
    early_stopping_threshold = 0.001,
    show_train_metrics=True,
    save_every=save_every,
    model_name=model_name,
    overwrite=False
)

In [ ]:
#loss
visualise_graph(training_data=train_loss,
                validation_data=val_loss,
                y_label='Loss',
                x_label='Epoch')

#rouge-1
rouge_train =  [rouge_dict['rouge1'] for rouge_dict in train_rouge]
rouge_val =  [rouge_dict['rouge1'] for rouge_dict in val_rouge]
visualise_graph(training_data=rouge_train,
                validation_data=rouge_val,
                y_label='Rouge-1 Score',
                x_label='Epoch')

#bleu4
bleu4_train =  [bleu_dict['BLEU4'] for bleu_dict in train_bleu]
bleu4_val =  [bleu_dict['BLEU4'] for bleu_dict in val_bleu]
visualise_graph(training_data=bleu4_train,
                validation_data=bleu4_val,
                y_label='BLEU-4 Score',
                x_label='Epoch')

#meteor
meteor_train =  [meteor_dict['meteor'] for meteor_dict in train_meteor]
meteor_val =  [meteor_dict['meteor'] for meteor_dict in val_meteor]
visualise_graph(training_data=meteor_train,
                validation_data=meteor_val,
                y_label='Meteor Score',
                x_label='Epoch')

In [ ]:
# Save the data into csv
data = zip(train_loss, bleu4_train, rouge_train, meteor_train, val_loss, bleu4_val, rouge_val, meteor_val)
dataframe = pd.DataFrame(data, columns=["Train Loss","Train BLEU", "Train ROUGE", "Train METEOR","Val Loss", "Val BLEU", "Val ROUGE", "Val METEOR"])
dataframe.to_csv(f"../resources/training_results/{model_name}.csv")

##### Finetuned resnet50

In [ ]:
# Clear GPU memory
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
finetuned_resnet = torch.load('../models/cnn/resnet_20_best.pt')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyper parameters
img_size = (224,224)
feature_size = 2048
embed_size = 256 
hidden_size = 512
vocab_size = len(dataset.vocab)
learning_rate = 3e-4
num_epochs = 75
save_every = 25
model_name = "AttnModel_Resnet20FT"


print(device)

In [ ]:
model = ResNetEncoderAttentionDecoder(finetuned_model=finetuned_resnet,
                                      feature_dim=feature_size,
                                      embedding_dim=embed_size, 
                                      hidden_dim=hidden_size, 
                                      vocab_size=vocab_size, 
                                      device=device,
                                      train_cnn=False,
                                      drop_prob=0.3,
                                      sample_temp=0.2).to(device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loss, train_bleu, train_rouge, train_meteor, val_loss, val_bleu, val_rouge, val_meteor=train(
    model=model,
    criterion=criterion,
    optimiser=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    image_size=img_size,
    transformer=False,
    batch_first=True,
    vocabulary=dataset.vocab,
    device=device,
    num_epochs=num_epochs,
    early_stopping_threshold = 0.001,
    show_train_metrics=True,
    save_every=save_every,
    model_name=model_name,
    overwrite=False
)

In [ ]:
#loss
visualise_graph(training_data=train_loss,
                validation_data=val_loss,
                y_label='Loss',
                x_label='Epoch')

#rouge-1
rouge_train =  [rouge_dict['rouge1'] for rouge_dict in train_rouge]
rouge_val =  [rouge_dict['rouge1'] for rouge_dict in val_rouge]
visualise_graph(training_data=rouge_train,
                validation_data=rouge_val,
                y_label='Rouge-1 Score',
                x_label='Epoch')

#bleu4
bleu4_train =  [bleu_dict['BLEU4'] for bleu_dict in train_bleu]
bleu4_val =  [bleu_dict['BLEU4'] for bleu_dict in val_bleu]
visualise_graph(training_data=bleu4_train,
                validation_data=bleu4_val,
                y_label='BLEU-4 Score',
                x_label='Epoch')

#meteor
meteor_train =  [meteor_dict['meteor'] for meteor_dict in train_meteor]
meteor_val =  [meteor_dict['meteor'] for meteor_dict in val_meteor]
visualise_graph(training_data=meteor_train,
                validation_data=meteor_val,
                y_label='Meteor Score',
                x_label='Epoch')

In [ ]:
# Save the data into csv
data = zip(train_loss, bleu4_train, rouge_train, meteor_train, val_loss, bleu4_val, rouge_val, meteor_val)
dataframe = pd.DataFrame(data, columns=["Train Loss","Train BLEU", "Train ROUGE", "Train METEOR","Val Loss", "Val BLEU", "Val ROUGE", "Val METEOR"])
dataframe.to_csv(f"../resources/training_results/{model_name}.csv")

##### Pretrained resnet50

In [ ]:
# Clear GPU memory
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyper parameters
img_size = (224,224)
feature_size = 2048
embed_size = 256 
hidden_size = 512
vocab_size = len(dataset.vocab)
learning_rate = 3e-4
num_epochs = 75
save_every = 5
model_name = "AttnModel_ResnetPT"


print(device)

In [ ]:
model = ResNetEncoderAttentionDecoder(finetuned_model=None,
                                      feature_dim=feature_size,
                                      embedding_dim=embed_size, 
                                      hidden_dim=hidden_size, 
                                      vocab_size=vocab_size, 
                                      device=device,
                                      train_cnn=False,
                                      drop_prob=0.3,
                                      sample_temp=0.2).to(device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loss, train_bleu, train_rouge, train_meteor, val_loss, val_bleu, val_rouge, val_meteor= train(
    model=model,
    criterion=criterion,
    optimiser=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    image_size=img_size,
    transformer=False,
    batch_first=True,
    vocabulary=dataset.vocab,
    device=device,
    num_epochs=num_epochs,
    early_stopping_threshold = 0.001,
    show_train_metrics=True,
    save_every=save_every,
    model_name=model_name,
    overwrite=False
)

In [ ]:
#loss
visualise_graph(training_data=train_loss,
                validation_data=val_loss,
                y_label='Loss',
                x_label='Epoch')

#rouge-1
rouge_train =  [rouge_dict['rouge1'] for rouge_dict in train_rouge]
rouge_val =  [rouge_dict['rouge1'] for rouge_dict in val_rouge]
visualise_graph(training_data=rouge_train,
                validation_data=rouge_val,
                y_label='Rouge-1 Score',
                x_label='Epoch')

#bleu4
bleu4_train =  [bleu_dict['BLEU4'] for bleu_dict in train_bleu]
bleu4_val =  [bleu_dict['BLEU4'] for bleu_dict in val_bleu]
visualise_graph(training_data=bleu4_train,
                validation_data=bleu4_val,
                y_label='BLEU-4 Score',
                x_label='Epoch')

#meteor
meteor_train =  [meteor_dict['meteor'] for meteor_dict in train_meteor]
meteor_val =  [meteor_dict['meteor'] for meteor_dict in val_meteor]
visualise_graph(training_data=meteor_train,
                validation_data=meteor_val,
                y_label='Meteor Score',
                x_label='Epoch')

In [ ]:
# Save the data into csv
data = zip(train_loss, bleu4_train, rouge_train, meteor_train, val_loss, bleu4_val, rouge_val, meteor_val)
dataframe = pd.DataFrame(data, columns=["Train Loss","Train BLEU", "Train ROUGE", "Train METEOR","Val Loss", "Val BLEU", "Val ROUGE", "Val METEOR"])
dataframe.to_csv(f"../resources/training_results/{model_name}.csv")

#### Evaluation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_size = 2048
embed_size = 256 
hidden_size = 512
vocab_size = len(dataset.vocab)

In [ ]:
finetuned_incept = torch.load('../models/cnn/incept_12_best.pt')
test_model1 = InceptV3EncoderAttentionDecoder(finetuned_model=finetuned_incept,
                                feature_dim=feature_size,
                                embedding_dim=embed_size, 
                                hidden_dim=hidden_size, 
                                vocab_size=vocab_size, 
                                device=device,
                                train_cnn=False,
                                drop_prob=0.3,
                                sample_temp=0.2).to(device)
test_criterion1 = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
test_model1.load_state_dict(torch.load('../models/image_captioning/AttnModel_Incept12FT.pt'))

In [ ]:
test_model2 = InceptV3EncoderAttentionDecoder(finetuned_model=None,
                                feature_dim=feature_size,
                                embedding_dim=embed_size, 
                                hidden_dim=hidden_size, 
                                vocab_size=vocab_size, 
                                device=device,
                                train_cnn=False,
                                drop_prob=0.3,
                                sample_temp=0.2).to(device)
test_criterion2 = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
test_model2.load_state_dict(torch.load('../models/image_captioning/AttnModel_InceptPT.pt'))

In [ ]:
finetuned_resnet = torch.load('../models/cnn/resnet_20_best.pt')
test_model3 = ResNetEncoderAttentionDecoder(finetuned_model=finetuned_resnet,
                                      feature_dim=feature_size,
                                      embedding_dim=embed_size, 
                                      hidden_dim=hidden_size, 
                                      vocab_size=vocab_size, 
                                      device=device,
                                      train_cnn=False,
                                      drop_prob=0.3,
                                      sample_temp=0.2).to(device)
test_criterion3 = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
test_model3.load_state_dict(torch.load('../models/image_captioning/AttnModel_Resnet20FT.pt'))

In [ ]:
test_model4 = ResNetEncoderAttentionDecoder(finetuned_model=None,
                                      feature_dim=feature_size,
                                      embedding_dim=embed_size, 
                                      hidden_dim=hidden_size, 
                                      vocab_size=vocab_size, 
                                      device=device,
                                      train_cnn=False,
                                      drop_prob=0.3,
                                      sample_temp=0.2).to(device)
test_criterion4 = torch.nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
test_model4.load_state_dict(torch.load('../models/image_captioning/AttnModel_ResnetPT.pt'))

In [ ]:
all_models = [[test_model1, test_criterion1, (299,299)],
              [test_model2, test_criterion2, (299,299)],
              [test_model3, test_criterion3, (224,224)],
              [test_model4, test_criterion4, (224,224)]]


for idx, models in enumerate(all_models):
    test_loss, test_bleu, test_rouge, test_meteor = eval(model=models[0],
                                criterion=models[1],
                                dataloader=test_dataloader,
                                image_size=models[2],
                                transformer=False,
                                batch_first=True,
                                vocabulary=test_dataset.vocab,
                                device=device
                                )
    
    print(f"Test Model {idx}\nTest Loss: {test_loss}\nTest BLEU:{test_bleu}\nTest Rouge:{test_rouge}\nTest Meteor:{test_meteor}\n----------------------------------------")


### 5. Application of model inference & multiple model inference
NOTE: the generate captions function should belong to your model class and have your own implementation depending on your model architecture
<br/><br/>
For easy standarisation you please make your `caption_image` function in your class have the following:
1. Inputs: image, vocabulary, device, max_length
2. Outputs: string prediction, attention (or None) 



#### Imports

In [ ]:
#Assuming mean and std are defined as follows:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def unnormalize(image:np.array, mean:np.array, std:np.array):
    """
    Function to unnormalize an image given its mean and std
    
    Args: 
        image (np.array): Numpy array of the image
        mean (np.array): Numpy array of the mean 
        std (np.array): Numpy array of the std

    Returns:
        Unnormalised numpy array of the image
    """

    for t, m, s in zip(image, mean, std):
        t.mul_(s).add_(m)    # for inplace operations
    return image


def caption_image(model, 
                  dataloader, 
                  image_size:tuple,
                  vocabulary:Vocabulary, 
                  device:str, 
                  mean:np.array=np.array([0.485, 0.456, 0.406]), 
                  std:np.array=np.array([0.229, 0.224, 0.225]), 
                  num_batches:int=1, 
                  num_images:int=5, 
                  max_length:int=50, 
                  show_plot:bool=False):
    """
    Function to generate model predictions from a dataloader

    Arg:
        model: model to general model prediction (ensure that your model has the function caption_image)
        dataloader: dataset to generate prediction
        image_size (tuple): image size of images for the model
        vocabulary (Vocabulary): dataset vocabulary
        device (str): cpu or cuda,
        mean (np.array): Numpy array of the mean used for normalisation
        std (np.array): Numpy array of the std used for normalisation
        num_batches (int, optional): how many batches iterating from dataloader, defaults to 1
        num_image (int, optional): how many images per batch to generate model prediction, defaults to 5
        max_length (int, optional): maximum length of generated captions, defaults to 50
        show_plot (bool, optional): show the image and generated captions in a plot, defaults to False
    
    Returns:
        all_predictions (dict): Dictionary containing the list of all generated captions and actual captions
    
    """
    model.eval()
    #dictionary containing all the generated predictions and actual predictions
    all_predictions = {'Predicted': [], 'Possible Captions': []}

    #iterate for num of batches we are testing
    for j in range(num_batches):
        #load images from dataloader
        features, annotations, all_annotations = next(iter(dataloader))

        #take first k from batch
        for i in range(num_images):
            features = torch.nn.functional.interpolate(features, size=image_size, mode='bilinear') #resize image for model, using same as transforms.resize()
            image = features[i].unsqueeze(0).to(device)
            
            #generate captions from model
            generated_caption, attention = model.caption_image(image, vocabulary, device, max_length=max_length)
            
            #plot image and captions
            if show_plot:
                fig, ax = plt.subplots(figsize=(5, 5))
                img = features[i].squeeze()
                img = unnormalize(img, mean, std)  # Unnormalize the image
                img = np.transpose(img.numpy(), (1, 2, 0))
                ax.imshow(img)
                ax.axis('off')
                ax.set_title(f'Model Prediction:\n{" ".join(generated_caption[1:-1])}\n\nAll Possible Predictions:\n' + "\n".join(all_annotations[i]), loc='left')

                plt.tight_layout()
                plt.show()

            all_predictions['Predicted'].append(generated_caption)
            all_predictions['Possible Captions'].append(all_annotations[i])

    return all_predictions


def multiple_model_captions(
        model_list:list[str, tuple],
        dataloader, 
        vocabulary:Vocabulary, 
        device:str, 
        mean:np.array=np.array([0.485, 0.456, 0.406]), 
        std:np.array=np.array([0.229, 0.224, 0.225]), 
        num_batches:int=1, 
        num_images:int=5, 
        max_length:int=50, 
        show_plot:bool=False):
    """
    Function to generate model predictions for multiple models from the same dataloader

    Arg:
        model_list (list[str, tuple]): list of [model, img_size] to general model prediction (ensure that your model has the function caption_image)
            eg. [[model1, (224,224)], [model2, (256,256)]]
        dataloader: dataset to generate prediction
        vocabulary (Vocabulary): dataset vocabulary
        device (str): cpu or cuda,
        mean (np.array): Numpy array of the mean used for normalisation
        std (np.array): Numpy array of the std used for normalisation
        num_batches (int, optional): how many batches iterating from dataloader, defaults to 1
        num_image (int, optional): how many images per batch to generate model prediction, defaults to 5
        max_length (int, optional): maximum length of generated captions, defaults to 50
        show_plot (bool, optional): show the image and generated captions in a plot, defaults to False
    
    Returns:
        all_predictions (dict): Dictionary containing the dictionary of all generated captions for each model and list of actual captions
    
    """
    for model, img_size in model_list:
        model.eval()

    #dictionary containing all the generated predictions and actual predictions
    all_predictions = {'Predicted': {}, 'Possible Captions': []}

    #iterate for num of batches we are testing
    for j in range(num_batches):
        #load images from dataloader
        features, annotations, all_annotations = next(iter(dataloader))
        
        #take first k from batch
        for i in range(num_images):
            image = features[i].unsqueeze(0).to(device)
            all_captions = []

            for idx, (model, img_size) in enumerate(model_list):
                #resize image
                image = torch.nn.functional.interpolate(image, size=img_size, mode='bilinear') #resize image
                #generate captions from model
                generated_caption, attention = model.caption_image(image, vocabulary, device, max_length=max_length)
                all_captions.append(" ".join(generated_caption[1:-1]))
                model_predictions = all_predictions['Predicted'].get(f"model_{idx}", [])
                model_predictions.append(all_captions)
                all_predictions['Predicted'][f"model_{idx}"] = model_predictions

            #plot image and captions
            if show_plot:
                fig, ax = plt.subplots(figsize=(5, 5))
                img = features[i].squeeze()
                img = unnormalize(img, mean, std)  # Unnormalize the image
                img = np.transpose(img.numpy(), (1, 2, 0))
                ax.imshow(img)
                ax.axis('off')

                pred = '\n'.join(all_captions)
                annotation = '\n'.join(all_annotations[i])

                ax.set_title(f'All Model Predictions:\n{pred}\n\nAll Possible Predictions:\n{annotation}', loc='left')

                plt.tight_layout()
                plt.show()

            all_predictions['Possible Captions'].append(all_annotations[i])

    return all_predictions

#### Inference

##### Multi-model

In [ ]:
model_list = [[test_model1, (299,299)],
              [test_model2, (299,299)],
              [test_model3, (224,224)],
              [test_model4, (224,224)]]

In [ ]:
predictions = multiple_model_captions(
    model_list=model_list,
    dataloader=test_dataloader,
    vocabulary= test_dataset.vocab,
    device=device,
    mean=mean,
    std=std,
    num_batches=10,
    num_images=1,
    max_length=50,
    show_plot=True
)

##### Single Model

In [ ]:
predictions = caption_image(
    model=test_model1,
    dataloader=test_dataloader,
    image_size=(299,299),
    vocabulary=test_dataset.vocab,
    device=device,
    mean=mean,
    std=std,
    num_batches=1,
    num_images=10,
    max_length=50,
    show_plot=True
)